In [2]:
import optuna
from skimage import filters
from scipy.ndimage import gaussian_filter
from sklearn.metrics import f1_score
import colorcorrect.algorithm as cca
import mahotas as mh
import cellseg.src.image_worker as iw
import cellseg.src.convert_worker as cw
import cellseg.src.shape_descriptors as sd
import numpy as np

img = mh.imread("cellseg/Images/all_images/2023_12_14_image_006.png")

file_path = "cellseg/Labels/orig/cytoplasm/2023_12_14_image_006.npy"

cytoplasm_labels = np.load(file_path)

In [3]:
def morph_cyto(
    img: np.ndarray,
    mask_size: int,
    iterations: int,
    min_size: int,
    slope: int,
    limit: int,
    samples: int,
    subwidth: int,
    subheight: int,
    sigma: float,
    label: np.ndarray | None = None,
) -> np.ndarray:
    """watershed

    If label is provided, it will calculate the f1 score.

    Args:
        img (np.ndarray): Original image
        mask_size (int): Mask for noise removal
        iterations (int): Number of iterations for noise removal
        min_size (int): Minimum size of region to keep
        slope (int): Slope for color correction
        limit (int): Limit for color correction
        samples (int): Samples for color correction
        subwidth (int): Subwidth for color correction
        subheight (int): Subheight for color correction
        sigma (float): Sigma for gaussian filter
        label (np.ndarray, optional): Ground truth labels. Defaults to None.

    Returns:
        np.ndarray:
    """
    img_c_corrected = cca.automatic_color_equalization(
        img, slope=slope, limit=limit, samples=samples
    )

    img_blurred = gaussian_filter(img_c_corrected, sigma=sigma)

    RGB_balanced = cca.luminance_weighted_gray_world(
        img_blurred, subwidth=subwidth, subheight=subheight
    )
    r, g, b = RGB_balanced[:, :, 0], RGB_balanced[:, :, 1], RGB_balanced[:, :, 2]

    thresholds = filters.threshold_multiotsu(g, classes=3)
    regions = np.digitize(g, bins=thresholds)
    bin_cyto_nuclei = cw.convert_labeled_to_bin(regions, background=2)
    bin_cyto_nuclei = iw.close_holes_remove_noise(
        bin_cyto_nuclei, mask_size=mask_size, iterations=iterations
    )

    img_labeled_cytoplasm, nr_cytoplasm = mh.label(bin_cyto_nuclei)
    img_labeled_cytoplasm = iw.remove_small_regions(img_labeled_cytoplasm, min_size=min_size)
    img_cytoplasm_nuclei = mh.labeled.remove_bordering(img_labeled_cytoplasm)

    if label is not None:
        f1 = f1_score(label.flatten(), img_cytoplasm_nuclei.flatten(), average="micro")
        return img_cytoplasm_nuclei, f1

    return img_cytoplasm_nuclei, None

In [ ]:
from pathlib import Path

def objective(trial):

    imgs_dir = Path("cellseg/Images/all_images/")

    # Hyperparameters to optimize
    slope = trial.suggest_int("slope", 5, 20)
    limit = trial.suggest_int("limit", 500, 2000)
    samples = trial.suggest_int("samples", 100, 1000)
    sigma = trial.suggest_float("sigma", 1.0, 15.0)
    subwidth = trial.suggest_int("subwidth", 10, 30)
    subheight = trial.suggest_int("subheight", 10, 30)
    mask_size = trial.suggest_int("mask_size", 2, 5)
    iterations = trial.suggest_int("iterations", 1, 10)
    min_size = trial.suggest_int("min_size", 50, 300)
    
    scores = []

    for img_path in imgs_dir.glob("*.png"):
    _, f1 = morph_cyto(
        img,
        mask_size=mask_size,
        iterations=iterations,
        min_size=min_size,
        slope=slope,
        limit=limit,
        samples=samples,
        subwidth=subwidth,
        subheight=subheight,
        sigma=sigma,
        label=cytoplasm_labels,
    )

    return f1

In [5]:
study = optuna.create_study(
    storage="sqlite:///optuna.db",
    load_if_exists=True,
    study_name="cytoplasm_segmentation",
    direction="maximize",
)
study.optimize(objective, n_trials=1)

[I 2024-11-19 20:20:07,772] Using an existing study with name 'cytoplasm_segmentation' instead of creating a new one.
[I 2024-11-19 20:20:40,410] Trial 32 finished with value: 0.7769211476856909 and parameters: {'slope': 10, 'limit': 1898, 'samples': 806, 'sigma': 7.393631669526211, 'subwidth': 17, 'subheight': 21, 'mask_size': 3, 'iterations': 8, 'min_size': 73}. Best is trial 12 with value: 0.7882117702218835.


In [19]:
print("Best parameters:", study.best_params)
print("Best F1 score:", study.best_value)

Best parameters: {'slope': 5, 'limit': 1958, 'samples': 646, 'sigma': 1.006495976340624, 'subwidth': 10, 'subheight': 18, 'mask_size': 5, 'iterations': 9, 'min_size': 140}
Best F1 score: 0.7882117702218835
